In [0]:
class PostingCompress:
  def __init__(self,mode):
    _dict = {"Gama":self.__toGama,"ContinueBit":self.__toContinueBit}
    self.mode=_dict[mode]
  
  def execute(self,posting):
    comp_posting=[]
    for linkList in posting:
      new_coding=str (self.mode(linkList[0]))
      for i,item in enumerate(linkList[1:]):
        new_coding += str (self.mode(item - linkList[i]))
      comp_posting.append(new_coding)
    return comp_posting,self.Decompress

  def Decompress(self,comp_posting):
    posting = list()
    if self.mode == self.__toGama:
      func = self.__fromGama
    elif self.mode == self.__toContinueBit:
      func = self.__fromContinueBit
    for binary in comp_posting:
      posting.append(func(binary))
    
    return posting

  def __fromGama(self,binary):
    unary = True
    offset = False
    posting=list()
    count=0
    for i,bit in enumerate(binary):
      if unary:
        if bit == "1" :
          count += 1
          continue
        elif bit == "0" and i == 0:#delete plus one
          posting.append(int("1",2) -1)
          continue
        elif bit == "0" and i !=0:
          unary = False
          offset = True
      if offset:
        try: #delete plus one    
          posting.append((posting[-1] + int("1" + binary[i+1:i+count+1],2)) -1)
        except:#delete plus one 
          posting.append(int("1" + binary[i+1:i+count+1],2) -1)
        offset =False
      
      if count != 0:
        count -= 1
        continue 
      else :
        count=0
        unary = True
    return posting
          

  def __fromContinueBit(self,binary):
    posting=list()
    for i in range(len(binary) // 8) :
      try:
        posting.append(posting[-1] + int(binary[(i)*8 +1:(i+1)*8],2))
      except:
        posting.append(int(binary[(i)*8 +1:(i+1)*8],2))
    return posting



  def __toGama(self,dec):
    #we should not have zero in GAMA .
    size=len(bin(dec +1)[2:]) -1 # +1 <----
    Gama = size * "1" + "0" + bin(dec + 1)[3:]
    return Gama

  def __toContinueBit(self,dec):
    dec=bin(dec)[2:]
    result=''
    while (len(dec) >= 7):
      temp=dec[-7:]
      dec=dec[:-7]
      result = "1" + temp + result
    res = result
    if len(result) and len(dec) != 0:   
      res= "0" + ((7 - len(dec)) *"0") + dec + result
    elif len(dec) != 0 and len(result) == 0:
      res= "1" + ((7 - len(dec)) *"0") + dec + result
    return res 



pc=PostingCompress("Gama")
x=pc.execute([[0,13,54,87,110],[1,5,7,18,38]])
print(x)
pc.Decompress(x)